In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import trapezoid, simpson
from scipy.interpolate import lagrange
from numpy.polynomial.legendre import leggauss

## Zadanie 1

In [2]:
vs = np.array([0, 1.0, 1.8, 2.4, 3.5, 4.4, 5.1, 6.0])
Ps = np.array([0, 4.7, 12.2, 19.0, 31.8, 40.1, 43.8, 43.2]) * 1000
m = 2000
v1 = 1.0
v2 = 6.0
N = 100

v_axis = np.linspace(v1, v2, 1000)

In [3]:
def my_lagrange(x, X, Y):
    n = len(X)
    y = 0
    for i in range(n):
        w = 1.0
        for j in range(n):
            if i != j:
                w *= (x-X[j])/(X[i]-X[j])
        y += w*Y[i]
    return y

In [4]:
def f(v):
    P = my_lagrange(v, vs, Ps)
    return v / P

In [5]:
def trapez(f, a, b, N):
    h = (b - a) / N
    x = np.linspace(a, b, N + 1)
    y = f(x)
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])


In [6]:
def simps(f, a, b, N):
    if N % 2 != 0:
        raise ValueError("N must be even number")

    h = (b - a) / N
    x = np.linspace(a, b, N + 1)
    y = f(x)

    return h / 3 * (y[0] + y[-1] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]))


In [7]:
I_trapez = m*trapez(f, v1, v2, N)
print(I_trapez)
I_simp = m*simps(f, v1, v2, N)
print(I_simp)

1.2785753265449442
1.2785007577364391


In [8]:
f_scipy = vs[1:] / Ps[1:]

I_trapez_scipy = m*trapezoid(f_scipy, vs[1:])
print(I_trapez_scipy)
I_simpson_scipy = m*simpson(y=f_scipy, x=vs[1:])
print(I_simpson_scipy)

1.2984952383952837
1.282121251477293


## Zadanie 2

In [20]:
def z2(x):
    return np.cos(2/np.cos(x))

a2 = -1
b2 = 1
N2 = [2, 4, 6]

In [30]:
Iz2_simpson = [simps(z2, a2, b2, i) for i in N2]
print(Iz2_simpson)

x2 = [np.linspace(a2, b2, i+1) for i in N2]
y2 = [z2(i) for i in x2]
Iz2_scipy = [simpson(y=y2[i], x=x2[i]) for i in range(3)]
print(Iz2_scipy)

[-1.1196854554190074, -1.2884087548681498, -1.339188110791998]
[-1.1196854554190074, -1.2884087548681498, -1.3391881107919983]


## Zadanie 3

Robimy podstawienie $x = t^{-1}$, wtedy

$$
    \int_{1}^{\infty}(1+x^4)^{-1}\text{d}x = \int_{0}^{1}\frac{t^2}{t^4+1}\text{d}t
$$

In [31]:
def z3(t):
    return t**2/(t**4+1)

a3 = 0
b3 = 1
N3 = 5

In [34]:
Iz3_trapez = trapez(z3, a3, b3, N3)
print(Iz3_trapez)

x3 = np.linspace(a3, b3, N3+1)
y3 = z3(x3) 
Iz3_scipy = trapezoid(y3, x3)
print(Iz3_scipy)

0.24373374765139955
0.24373374765139955


## Zadanie 4

In [47]:
theta0 = np.array([15*np.pi/180, 30*np.pi/180, 45*np.pi/180])
h_answers = np.zeros(len(theta0))
h_prop = np.zeros(len(theta0))
a4 = 0
b4 = np.pi/2
N4 = 50
for i in range(len(theta0)):
    def z4(theta):
        return 1/np.sqrt((1-np.sin(theta0[i]/2)*np.sin(theta))**2)
    
    h_answers[i] = simps(z4, a4, b4, N4)
    h_prop[i] = h_answers[i]/b4

print(h_answers)
print(h_prop)


[1.71637989 1.89724269 2.12527212]
[1.09268137 1.20782221 1.35299025]


## Zadanie 5

In [48]:
def z5(x):
    return np.log(x)/(x**2 - 2*x + 2)

Wielomian Legendre'a
$$
P_{n}(x) = \frac{2n-1}{n}xP_{n-1}(x) - \frac{n-1}{n}P_{n-2}(x)
$$
Pochodna wielomianu Legendre'a
$$
P'_n(x) = \frac{n}{x^2-1}(xP_n(x)- P_{n-1}(x))
$$

In [62]:
def legendre(n, x):
    if n == 0:
        return 1.0
    if n == 1:
        return x

    Pnm1 = x
    Pnm2 = 1.0

    for k in range(2, n + 1):
        Pn = ((2*k - 1)*x*Pnm1 - (k - 1)*Pnm2) / k
        Pnm2 = Pnm1
        Pnm1 = Pn

    return Pn

def dlegendre(n, x):
    return n / (x**2 - 1) * (x*legendre(n, x) - legendre(n - 1, x))


def gauss_legendre(f, a, b, N, eps=1e-14):
    xk = []
    Ak = []

    for k in range(N):
        x = np.cos(np.pi * (k + 0.75) / (N + 0.5))

        while True:
            dx = -legendre(N, x) / dlegendre(N, x)
            x += dx
            if abs(dx) < eps:
                break

        xk.append(x)

        w = -2 / ((N+1) * legendre(N+1, x) * dlegendre(N, x))
        Ak.append(w)

    result = 0.0
    for i in range(N):
        t = 0.5*(b - a)*xk[i] + 0.5*(b + a)
        result += Ak[i] * f(t)

    return 0.5*(b - a) * result



In [65]:
N5 = np.array([2, 4])
a5 = 1
b5 = np.pi

Iz5 = [gauss_legendre(z5, a5, b5, i) for i in N5]
print(Iz5)

[0.6067250228624488, 0.5847680362127089]


## Zadanie 6

In [27]:
def f1(x):
    return x**3 - 2*x

def f2(x):
    return np.sin(x)

def f3(x):
    return np.exp(x)

def df1(x):
    return 3*x**2 - 2

def df2(x):
    return np.cos(x)

def df3(x):
    return np.exp(x)

h = np.array([0.1, 0.01, 0.001])

def Df1(f, x, h):
    return (f(x+h) - f(x))/h

def Dc2(f, x, h):
    return (f(x+h)-f(x-h))/(2*h)

def Dc4(f, x, h):
    return (4*Dc2(f, x, h) - Dc2(f, x, 2*h))/3

In [28]:
f1_prime_D = np.zeros((3, 3))
f2_prime_D = np.zeros((3, 3))
f3_prime_D = np.zeros((3, 3))
x_f1 = 1
x_f2 = np.pi/3
x_f3 = 0
for i in range(2):
    f1_prime_D[i][0] = df1(x_f1) - Df1(f1, x_f1, h[i])
    f1_prime_D[i][1] = df1(x_f1) - Dc2(f1, x_f1, h[i])
    f1_prime_D[i][2] = df1(x_f1) - Dc4(f1, x_f1, h[i])

    f2_prime_D[i][0] = df2(x_f2) - Df1(f2, x_f2, h[i])
    f2_prime_D[i][1] = df2(x_f2) - Dc2(f2, x_f2, h[i])
    f2_prime_D[i][2] = df2(x_f2) - Dc4(f2, x_f2, h[i])

    f3_prime_D[i][0] = df3(x_f3) - Df1(f3, x_f3, h[i])
    f3_prime_D[i][1] = df3(x_f3) - Dc2(f3, x_f3, h[i])
    f3_prime_D[i][2] = df3(x_f3) - Dc4(f3, x_f3, h[i])

print("f'(x) - Df1","|", "f'(x) - Dc2", "|", "f'(x) - Dc4")
print("f1")
print(f1_prime_D)
print("f2")
print(f2_prime_D)
print("f3")
print(f3_prime_D)


f'(x) - Df1 | f'(x) - Dc2 | f'(x) - Dc4
f1
[[-3.10000000e-01 -1.00000000e-02 -1.11022302e-15]
 [-3.01000000e-02 -1.00000000e-04 -4.66293670e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
f2
[[4.40981146e-02 8.32916766e-04 1.66468370e-06]
 [4.33842423e-03 8.33329166e-06 1.66658964e-10]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
f3
[[-5.17091808e-02 -1.66750020e-03  3.33730390e-06]
 [-5.01670842e-03 -1.66667500e-05  3.33348127e-10]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


## Zadanie 7

In [30]:
x7 = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
f7 = np.array([0.000000, 0.078348, 0.138910, 0.192916, 0.244981])

x0_index = 2
h7 = 0.1
Dc2_7 = (f7[x0_index+1] - f7[x0_index-1])/(2*h7)
Dc2_2h_7 = (f7[x0_index+2] - f7[x0_index-2])/(4*h7)
Dc4_7 = (4*Dc2_7 - Dc2_2h_7)/3

f_prime_02 = Dc4_7
print(f_prime_02)

0.5596358333333334


## Zadanie 8

In [7]:
x8 = np.array([-2.2, -0.3, 0.8, 1.9])
f8 = np.array([15.180, 10.962, 1.920, -2,.040])

In [8]:
def lagrange_coefficients(x, y):
    n = len(x)
    coeffs = np.zeros(n)

    for i in range(n):
        poly = np.poly1d([1])
        denom = 1
        for j in range(n):
            if i != j:
                poly *= np.poly1d([1, -x[j]])
                denom *= (x[i] - x[j])
        coeffs += y[i] * poly.coeffs / denom

    return coeffs

In [ ]:
f8_poly_coefs = lagrange_coefficients(x8, f8)
print(f8_poly_coefs)


[ 1.00403145 -0.29314654 -8.56540214  8.4458714 ]


In [16]:
def poly_derivative(w):
    deg = len(w) - 1
    dw = [w[i] * (deg - i) for i in range(deg)]
    return np.array(dw)

def calc_poly(coefs, x):
    w = 0
    for a in coefs:
        w = w * x + a
    return w


In [18]:
df8_poly_coefs = poly_derivative(f8_poly_coefs)
ddf8_poly_coefs = poly_derivative(df8_poly_coefs)
print(df8_poly_coefs)
print(ddf8_poly_coefs)

f8_prime_0 = calc_poly(df8_poly_coefs, 0)
f8_sec_0 = calc_poly(ddf8_poly_coefs, 0)
print(f8_prime_0, f8_sec_0)

[ 3.01209434 -0.58629309 -8.56540214]
[ 6.02418867 -0.58629309]
-8.565402136665993 -0.5862930860713557
